In [ ]:
import os
import scanpy as sc
import anndata as ad
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
out_dir = "output"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
sc.settings.verbosity = 3 
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

In [ ]:
# the following are needed to ensure that text in plt images opens as text box rather than glyphs in adobe
plt.rcParams['pdf.fonttype'] = 42  
plt.rcParams['ps.fonttype'] = 42  

plt.rcParams["figure.figsize"] = (4, 4)  

plt.rcParams["font.size"] = 14  
plt.rcParams["axes.labelsize"] = 16  
plt.rcParams["axes.titlesize"] = 18  
plt.rcParams["legend.fontsize"] = 14  
plt.rcParams["xtick.labelsize"] = 14  
plt.rcParams["ytick.labelsize"] = 14 


In [ ]:
base_path = "../Aligned_NucSeq/" # Folder with alignment output

# Dictionary :sample names as keys and relative paths as values
sample_names = {
    "nCN017K": "nCN017K/outs/filtered_feature_bc_matrix/",
    "nC018K": "nC018K/outs/filtered_feature_bc_matrix/",
    "nCN021K": "nCN021K/outs/filtered_feature_bc_matrix/",
    "nCN029K": "nCN029K/outs/filtered_feature_bc_matrix/",
    "nP072K": "nP072K/outs/filtered_feature_bc_matrix/",
    "nP073K": "nP073K/outs/filtered_feature_bc_matrix/",
    "nP103K": "nP103K/outs/filtered_feature_bc_matrix/",
    "nP105K": "nP105K/outs/filtered_feature_bc_matrix/",
    "nP108K": "nP108K/outs/filtered_feature_bc_matrix/",
    "nP126K": "nP126K/outs/filtered_feature_bc_matrix/",
    "nP137K": "nP137K/outs/filtered_feature_bc_matrix/",
    "nP139K": "nP139K/outs/filtered_feature_bc_matrix/",
    "nP140K": "nP140K/outs/filtered_feature_bc_matrix/",
    "nP143K": "nP143K/outs/filtered_feature_bc_matrix/",
    "nP144K": "nP144K/outs/filtered_feature_bc_matrix/"
}

samples = {key: os.path.join(base_path, value) for key, value in sample_names.items()}


In [ ]:
adatas = {}

for sample_id, filename in samples.items():
    sample_adata = sc.read_10x_mtx(filename, var_names='gene_symbols', cache=True) 
    sample_adata.var_names_make_unique()  
    adatas[sample_id] = sample_adata  # Store AnnData object in dictionary

print("Loaded samples:", list(adatas.keys()))


In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")

adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))

# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)


In [ ]:
sc.pp.filter_cells(adata, min_genes=50)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 10, :].copy()

In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata.raw = adata.copy()

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")
sc.pl.highly_variable_genes(adata)

In [ ]:
sc.pp.scale(adata) #This scales each gene to unit variance.

In [ ]:
sc.tl.pca(adata)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata,
    color= ["sample"],
    size=2,
)

In [ ]:
sc.external.pp.harmony_integrate(adata, key="sample", max_iter_harmony=50)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_pca_harmony")
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
sc.pl.umap(
    adata,
    color= ["sample", 'leiden'],
    size=2,
)

In [ ]:
adata.write("../nucseq_outs/adata_Nuc.h5ad")